### Парсер АВИТО

In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import random
import time
from tqdm import tqdm_notebook as tqdm
import re
import unicodedata
import collections

In [3]:
def clean(text):
    text = re.sub('\n', '', text)
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

def clean_info(text):
    text = unicodedata.normalize("NFKD", text)
    text = re.sub('(\n)+', ' ', text)
    text = re.sub('( ){2,}', ', ', text)
    text = text.strip()[2:-1]
    
    return text

def try_(soup, _tag, _class, _group='class'):
    try:
        item = soup.find(_tag, attrs={_group:_class}).text
    except:
        item = 'None'
    return item

def text(soup, url):
    text = ''
    text = 'Название объявления: ' + clean(try_(soup, 'span', 'title-info-title-text')) + '\n' + \
    'Информация об объявлении: ' + clean(try_(soup, 'div', 'title-info-metadata-item')) + '\n' + \
    'Информация об авторе: ' + clean_info(try_(soup, 'div', 'item-view-seller-info')) + '\n' + \
    'Адрес: ' + clean(try_(soup, 'div', 'item-map-location')).replace('Адрес: ', '').replace(' Посмотреть карту', '') + \
    '\n' + clean(try_(soup, 'div', 'item-params')) + '\n' + \
    'Цена кошки: ' + clean(try_(soup, 'span', 'price-value-string js-price-value-string')) + '\n' + \
    'Описание кошки: ' + clean(try_(soup, 'div', 'description', _group='itemprop')) + '\n' + \
    'URL объявления: ' + url
    
    return text

In [ ]:
for x in tqdm(range(40, 70)):
    url = 'https://www.avito.ru/rossiya/koshki?p=' + str(x) + '&view=list'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    articles = soup.find_all('a', attrs={'class':'description-title-link'})
    time.sleep(random.uniform(0, 10))
    for item in articles:
        url_of_article = item.get('href')
        begin = 'https://www.avito.ru'
        clean_url = begin + url_of_article
        req = requests.get(clean_url)
        soup_page = BeautifulSoup(req.text, "lxml")
        text = text(soup_page, clean_url)
        with open('/Users/alinashaymardanova/Downloads/avito_cats/' + url_of_article.replace('/', '_')[1:] + ".txt", "w") as f:
            f.write(text)
        time.sleep(random.uniform(0, 10))

A Jupyter Widget